In [5]:
search_for = 121
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002460479736328125
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 15
all cases: 237758115
type: [1, 1, 1, 1, 121] 121
cases of this type: 214358881
100000 12.007586788384852
200000 18.889284567914107
300000 17.168134196145417
400000 15.95755339805861
500000 17.483127464113256
600000 17.60146199232793
700000 16.74761293783721
800000 15.70158731471428
900000 9.843208234923065
1000000 14.253179515022746
1100000 19.400210514392214
1200000 19.247384618341265
1300000 17.29859588794536
1400000 16.997576130318066
1500000 16.631281287253262
1600000 16.694213602945492
1700000 16.530770792864114
1800000 8.613975198005031
1900000 16.121956336995094
2000000 19.734278086304272
2100000 18.673673478984146
2200000 20.482189043673458
2300000 18.014684302823603
2400000 17.956634208915908
2500000 18.904564341626347
2600000 19.35383416789752
2700000 7.693222771208008
2800000 20.327034274102225
2900000 19.38894048074279
3000000 

29400000 13.168142262578685
29500000 14.023641999647591
29600000 15.123357619833593
29700000 14.428808550499639
29800000 16.01377613385807
29900000 8.615408268422563
30000000 14.679012616522728
30100000 16.10670967801218
30200000 16.153644101513535
30300000 16.451126858155877
30400000 8.365628022857354
30500000 15.583781948328955
30600000 16.468817774308533
30700000 14.774537273894149
30800000 15.11251849905189
30900000 13.956697330320328
31000000 9.878584611872842
31100000 10.040286126724919
31200000 14.239743619613337
31300000 14.85753544838722
31400000 15.06344087418135
31500000 15.728009771139524
31600000 16.72516522038509
31700000 8.764922726863198
31800000 16.013815238313885
31900000 16.00515610449577
32000000 15.35463937931111
32100000 15.62302706646819
32200000 7.730504406708337
32300000 16.864161956111506
32400000 14.875114558662414
32500000 15.58138330396082
32600000 14.097365582174284
32700000 14.423924888983102
32800000 6.890096182762835
32900000 13.017623557522178
33000000

59100000 11.494455271608631
59200000 11.652228331154154
59300000 11.925901619542287
59400000 5.083015033399463
59500000 11.973601139820556
59600000 12.154860693601284
59700000 10.918214339167447
59800000 6.685723447863969
59900000 12.372086457720723
60000000 11.969570406671167
60100000 12.680302391410676
60200000 13.467380502193391
60300000 12.793935073208711
60400000 13.67940702595734
60500000 13.999621493358033
60600000 12.582178654998721
60700000 12.537110492406539
60800000 5.512461031182667
60900000 11.48432750807482
61000000 12.924118125084558
61100000 10.598864668225925
61200000 6.867496532832791
61300000 11.48856708376278
61400000 11.528825253807932
61500000 9.477737668655754
61600000 9.53910575455058
61700000 12.070307365172184
61800000 12.445466095816125
61900000 12.394179866331086
62000000 13.12038219773564
62100000 13.117637651091984
62200000 14.10878688919896
62300000 13.51020201071779
62400000 12.498884897297144
62500000 10.817593444241167
62600000 6.597026978298989
627000

88800000 10.061565097829432
88900000 10.111772886512155
89000000 9.629514124005778
89100000 9.764922350172988
89200000 9.865862351607095
89300000 8.178812041766012
89400000 6.2926206980091335
89500000 4.147122068656236
89600000 9.168813900012308
89700000 6.526989321998
89800000 9.874172562957526
89900000 9.896440242504859
90000000 9.230288517666628
90100000 9.316956993086151
90200000 10.454497448867185
90300000 10.668759077296318
90400000 10.146703931544693
90500000 10.622382218560077
90600000 9.735206776646693
90700000 9.795368635675397
90800000 10.274582148337219
90900000 9.562265952999025
91000000 9.895503654007674
91100000 7.720269996886208
91200000 7.58171873917222
91300000 4.591076160067896
91400000 6.21250043064837
91500000 9.732805980138798
91600000 9.538449290285085
91700000 9.671684576156782
91800000 10.03960209452073
91900000 9.71035804961913
92000000 9.623702828658352
92100000 11.447122460730313
92200000 10.76728706269256
92300000 9.926364274197388
92400000 10.2938468575250

118200000 8.03782280054754
118300000 7.660333599120087
118400000 8.270035841278705
118500000 7.834470298323264
118600000 8.01238930815662
118700000 8.78999225606554
118800000 9.081673489546152
118900000 8.515388478716067
119000000 8.009404644607445
119100000 8.374467172304929
119200000 8.406939540858527
119300000 8.14473293077209
119400000 7.755475916192214
119500000 5.188445358958897
119600000 3.642249367174476
119700000 3.9367432265663544
119800000 7.064921955838792
119900000 8.155549405019551
120000000 8.008198659689187
120100000 7.977648141529401
120200000 8.54514578619598
120300000 7.731898394586484
120400000 8.492449527444622
120500000 8.862713697517574
120600000 8.489823451803515
120700000 8.350733271807048
120800000 8.245530181312532
120900000 8.58046120996961
121000000 7.8365103447172375
121100000 8.098201595673693
121200000 7.437019164840331
121300000 4.350375926233907
121400000 3.5536392593480413
121500000 4.731625805947284
121600000 7.236190292789343
121700000 8.18112514434

147500000 5.756035511857784
147600000 4.059511920660387
147700000 5.0849361841284
147800000 5.992375085982227
147900000 5.772531545873374
148000000 3.023713732772274
148100000 5.790851666144851
148200000 5.835829578295768
148300000 2.7392209980562
148400000 5.90134880794149
148500000 6.590421104145169
148600000 6.499192340834539
148700000 6.492455301588512
148800000 6.734562115478307
148900000 6.696446247037719
149000000 6.695646103169378
149100000 6.427178470430026
149200000 6.147768674725628
149300000 5.999711793160234
149400000 3.3978901517497793
149500000 5.890578726063606
149600000 5.915617633601546
149700000 4.453371854303121
149800000 4.731381566241652
149900000 6.268149830930574
150000000 5.789804190239012
150100000 3.2459773417280315
150200000 6.05234956581184
150300000 6.134370760579556
150400000 6.648163063604941
150500000 6.782980922570334
150600000 6.616970524301797
150700000 6.506648092250026
150800000 6.727443073170487
150900000 6.1652992614830335
151000000 6.25505590868

176700000 4.242491625562161
176800000 4.502498038212859
176900000 2.2325053766927287
177000000 4.689322873036912
177100000 4.364827889505532
177200000 4.5635452121231195
177300000 4.812173165804973
177400000 4.35260370901172
177500000 2.376954629647785
177600000 4.530423124576718
177700000 4.038681523081991
177800000 3.9845111305453447
177900000 4.0228647112275855
178000000 4.02770236281278
178100000 1.6999323728710511
178200000 3.7645358351193066
178300000 3.8642024603358673
178400000 4.058599950280809
178500000 4.084833842928976
178600000 3.984783343742583
178700000 2.409721630621877
178800000 4.158921695026258
178900000 4.242036095601414
179000000 4.239916560970723
179100000 4.230823206268837
179200000 3.61760185155896
179300000 3.1202546431138187
179400000 4.18254935468036
179500000 4.087062215800424
179600000 3.656316288906965
179700000 3.8881689567366835
179800000 3.311624128608892
179900000 2.61167972442424
180000000 3.7916410124380886
180100000 4.001250669127279
180200000 3.804

205500000 1.6258679005212489
205600000 1.3071868938668305
205700000 2.5230005307919487
205800000 2.3839553897422454
205900000 2.4010834368497598
206000000 2.3827615110997624
206100000 2.3000187306277953
206200000 2.3702571924912066
206300000 2.406102740739842
206400000 1.375358503893584
206500000 2.0170232210792802
206600000 2.2868654062984355
206700000 2.288662924970875
206800000 2.2894793504426514
206900000 2.586560959191859
207000000 2.3949940862738495
207100000 2.365677698394885
207200000 1.8397633543773593
207300000 1.4776616501960458
207400000 1.3677830011381604
207500000 2.3077696224289306
207600000 2.2923839335364304
207700000 2.334837841896362
207800000 2.3558086881053217
207900000 2.3237594864377282
208000000 2.176778884909749
208100000 2.327912300952577
208200000 0.9379547516525388
208300000 2.4536353081123132
208400000 2.3184597216708163
208500000 2.201338682381252
208600000 2.3876004693137274
208700000 2.409761090277751
208800000 2.283118488492956
208900000 2.1718101353518

233400000 0.15001702136683465
233500000 0.1373458851423131
233600000 0.1461363291243083
233700000 0.1269094039569199
233800000 0.12888305364468694
best so far: 0
type: [1, 1, 1, 121, 1] 121
cases of this type: 1771561
233900000 0.07491194509454568
234000000 0.02498550936689973
234100000 0.024356634849942393
234200000 0.024006387770752112
234300000 0.024986793885827065
234400000 0.02436492311794228
234500000 0.021690470361202953
234600000 0.02088758939939737
234700000 0.021681574532131355
234800000 0.01897392615189155
234900000 0.018475608375181755
235000000 0.018609128176682525
235100000 0.016965594497972067
235200000 0.016386647490262987
235300000 0.01565130156720678
235400000 0.015131442126893335
235500000 0.015211569762885572
235600000 0.01404927025831408
best so far: 0
type: [1, 1, 11, 1, 11] 121
cases of this type: 1771561
235700000 0.06075421974905332
235800000 0.060142654048035545
235900000 0.05502181371010675
236000000 0.05418706537318892
236100000 0.050121510462969546
23620000